In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/2.part'):
    shutil.rmtree('./file/2.part')

os.makedirs('./file/2.part')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import re

# = = = = = = = = = = = = = = =

import numpy as np

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/1.part_number'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/1.part_number/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                data = {'dopartsearch': '1',
                        'partsearch[partnum][partsearch_007]': input_.loc[a, 'Part Number'],
                        'partsearch[manufacturer][partsearch_007]': input_.loc[a, 'Manufacturer'],
                        'partsearch[partgroup][partsearch_007]': input_.loc[a, 'Group'],
                        'partsearch[parttype][partsearch_007]': input_.loc[a, 'Type Code'],
                        'partsearch[do][partsearch_007]': '1',
                        'func': 'sendparttabsearch',
                        'giveback': json.dumps({}),
                        'api_json_request': '1'}

                b = 0
                while True:
                    b += 1
                    try:
                        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                             data=data,
                                             headers=get_header(),
                                             proxies=get_proxy(),
                                             timeout=(10, 10))

                        if resp.status_code == 200:
                            json_ = resp.json()
                            
                            if 'redirect_to_url' not in json_ and not resp.json()['giveback'] and 'collected_javascript' in json_:
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue
                            
                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(json_['searchnoderesults'], 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                list_tbody = html.xpath('//tbody[contains(@id, "listingcontainer[__GIP__")]')
                
                # = = = = = = = = = = = = = = =

                list_list_note = [tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row-moreinfo-truck"]/span[@class="span-link-underline-remover"]') for tbody in list_tbody]
                list_note = [etree.tostring(list_note[0]).decode('utf-8') if list_note else '' for list_note in list_list_note]
                list_note = [re.sub(r'<.*?>', '', note) for note in list_note]
                list_note = [re.sub(r' +', ' ', note).strip() for note in list_note]

                # = = = = = = = = = = = = = = =

                list_dict_note = []
                list_span = [tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row"]/span[contains(@title, "Please make sure")]')[0] for tbody in list_tbody]
                
                for span in list_span:
                    dict_note = {}
                    list_span_2 = span.xpath('./span[@class="listing-footnote-text"]')
                    list_note_2 = [etree.tostring(span_2).decode('utf-8') for span_2 in list_span_2]
                    list_note_2 = [re.sub(r'<.*?>', '', note_2) for note_2 in list_note_2]
                    list_note_2 = [re.sub(r' +', ' ', note_2).strip() for note_2 in list_note_2]
                    for i, note_2 in zip(range(len(list_note_2)), list_note_2):
                        dict_note[str(i)] = note_2
                
                    list_dict_note.append(dict_note)
    
                # = = = = = = = = = = = = = = =

                list_json = [json.loads(tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/input[contains(@id, "listing_data_essential[__GIP__")]/@value')[0]) for tbody in list_tbody]

                # = = = = = = = = = = = = = = =

                array_index_type_code = np.where(np.array([json_['parttype'] for json_ in list_json]) == input_.loc[a, 'Type Code'])

                # = = = = = = = = = = = = = = =

                list_part_code = [json_['partkey'] for json_ in list_json]
                
                # = = = = = = = = = = = = = = =

                list_list_info = [tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]') for tbody in list_tbody]
                list_info = [list_info[0].xpath('./@href')[0].strip() if list_info else '' for list_info in list_list_info]
                list_info = [info if info.startswith('https://www.rockauto.com/en/moreinfo.php?') else f'''https://www.rockauto.com/en/moreinfo.php?pk={part_code}&cc=0&pt={input_.loc[a, 'Type Code']}&Lennon=1''' for info, part_code in zip(list_info, list_part_code)]

                # = = = = = = = = = = = = = = =

                list_json = [json.loads(tbody.xpath('./tr[1]/td/input[contains(@id, "listing_data_supplemental[__GIP__")]/@value')[0]) for tbody in list_tbody]

                # = = = = = = = = = = = = = = =

                array_index_part_number = np.where(np.array([json_['partnumber'] for json_ in list_json]) == input_.loc[a, 'Part Number'])

                # = = = = = = = = = = = = = = =

                array_index_manufacturer = np.where(np.array([json_['catalogname'] for json_ in list_json]) == input_.loc[a, 'Manufacturer'])

                # = = = = = = = = = = = = = = =

                array_index_part = np.intersect1d(np.intersect1d(array_index_type_code, array_index_part_number), array_index_manufacturer)

                # = = = = = = = = = = = = = = =

                if len(array_index_part) == 1:
                    index_part = array_index_part[0]
                else:
                    raise

                # = = = = = = = = = = = = = = =

                choose = list_json[index_part]['paramdesc'].strip()

                # = = = = = = = = = = = = = = =

                list_oem = [td.xpath('./span[contains(@title, "Replaces these Alternate")]/text()') for td in list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]')]

                # = = = = = = = = = = = = = = =

                list_ = []
                for oem in list_oem:
                    if oem:
                        list_ += oem[0].split(',')
                list_ = [oem.strip() for oem in list_]
                    
                while '' in list_:
                    list_.remove('')
                
                oem = ';'.join(sorted(list(set(list_))))

                # = = = = = = = = = = = = = = =

                list_src = list_tbody[index_part].xpath('./tr[1]/td/div[contains(@class, "listing-image-space-occupy")]/input[contains(@id, "jsninlineimg[__GIP__")]')
                json_ = json.loads(list_src[0].xpath('./@value')[0]) if list_src else ''

                # = = = = = = = = = = = = = = =

                dict_src = {}
                if json_ != '':
                    list_src = json_['Slots']
                    for i, src in zip(range(len(list_src)), list_src):
                        dict_src[str(i)] = f'''https://www.rockauto.com{src['ImageData']['Full'].strip()}'''
                        
                # = = = = = = = = = = = = = = =

                price = list_tbody[index_part].xpath('./tr[1]/td[contains(@id, "listingtd[__GIP__") and contains(@id, "__][price]")]/span[contains(@id, "dprice[__GIP__") and contains(@id, "__][td]")]/span/span[contains(@id, "dprice[__GIP__") and contains(@id, "__][v]")]')
                price = etree.tostring(price[0]).decode('utf-8') if price else ''
                price = re.sub(r'<.*?>', '', price)
                price = re.sub(r' +', ' ', price).strip()
                dict_price = {'automatic': price}

                # = = = = = = = = = = = = = = =

                td = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]')[0]
                list_option = [etree.tostring(option).decode('utf-8') for option in td.xpath('./div/div[contains(@class, "listing-optionchoice-multiple-container")]/span[contains(@id, "ddrepl[optionchoice[__GIP__") and contains(@id, "__]][container]")]/span[@class="hide-if-js"]/select[contains(@id, "optionchoice[__GIP__") and contains(@id, "__]")]/option[@value!=""]')]
                list_option = [re.sub(r'<.*?>', '', option) for option in list_option]
                list_option = [re.sub(r' +', ' ', option).strip() for option in list_option]
                if list_option:
                    dict_price['choose'] = {}
                    for i, option in zip(range(len(list_option)), list_option):
                        dict_price['choose'][str(i)] = option

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Group': input_.loc[a, 'Group'],
                                         'Type': input_.loc[a, 'Type'],
                                         'Type Code': input_.loc[a, 'Type Code'],
                                         'Manufacturer': input_.loc[a, 'Manufacturer'],
                                         'Part Number': input_.loc[a, 'Part Number'],
                                         'Note 1': list_note[index_part],
                                         'Json_Note 2': json.dumps(list_dict_note[index_part]),
                                         'Choose': choose,
                                         'Json_Price': json.dumps(dict_price),
                                         'Vehicle': '',
                                         'OEM': oem,
                                         'Picture': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Info': list_info[index_part],
                                         'Json_Src': json.dumps(dict_src),
                                         'Part Code': list_part_code[index_part],
                                         'JOIN_MPNTCPC': f'''{input_.loc[a, 'Manufacturer']};{input_.loc[a, 'Part Number']};{input_.loc[a, 'Type Code']};{list_part_code[index_part]}'''}])
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'Type']} > {input_.loc[a, 'Manufacturer']} > {input_.loc[a, 'No']}.{input_.loc[a, 'Part Number']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/2.part/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/2.part/{file}',
              f'''./file/2.part/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

[状态：ok，尝试次数：13] - 83.33% - crawler_5 > Ball Joint > MEVOTECH > 373.GK9755
[剩余数量：1000] - [当前时间：18:14:32]

[状态：ok，尝试次数：58] - 83.33% - crawler_5 > Ball Joint > MEVOTECH > 335.GK9551
[剩余数量：999] - [当前时间：18:14:32]

[状态：ok，尝试次数：162] - 83.33% - crawler_5 > Ball Joint > MEVOTECH > 257.GK9042
[剩余数量：998] - [当前时间：18:14:32]

[状态：ok，尝试次数：16] - 83.33% - crawler_5 > Ball Joint > MEVOTECH > 364.GK9700
[剩余数量：997] - [当前时间：18:14:33]

[状态：ok，尝试次数：62] - 83.33% - crawler_5 > Ball Joint > MEVOTECH > 332.GK9531
[剩余数量：996] - [当前时间：18:14:33]

[状态：ok，尝试次数：83] - 83.33% - crawler_5 > Ball Joint > MEVOTECH > 313.GK9452
[剩余数量：995] - [当前时间：18:14:33]

[状态：ok，尝试次数：9] - 83.33% - crawler_5 > Ball Joint > MEVOTECH > 350.GK9633
[剩余数量：994] - [当前时间：18:14:33]

[状态：ok，尝试次数：14] - 83.33% - crawler_5 > Ball Joint > MEVOTECH > 354.GK9643
[剩余数量：993] - [当前时间：18:14:33]

[状态：ok，尝试次数：432] - 83.33% - crawler_5 > Ball Joint > MEVOTECH > 95.GK7157
[剩余数量：992] - [当前时间：18:14:33]

[状态：ok，尝试次数：10] - 83.33% - crawler_5 > Ball Joint > MEVOTECH > 

Progress: 100%|██████████████████████████████| 6/6 [00:00<00:00, 5469.64it/s]

Done ~
